In [201]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [595]:
def _parser_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': 'inzhener smetchik', \
        'page': '1'
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
    }
    
    link = 'https://www.superjob.ru/vakansii/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'icMQ_ _1_Cht'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a', {'class': 'icMQ_'})[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'iJCa5'})
                        
            for item in vacancy_items:
                vacancy_date.append(_parser_item_superjob(item))
                
    return vacancy_date

In [707]:
def _parser_item_superjob(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find_all('div', {'class': '_3mfro PlM3e'})
    vacancy_date['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.findParent() \
                            .find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    vacancy_date['company_name'] = company_name
    
    # city
    company_location = item.find('span',{' '})
    
    vacancy_date['city'] = company_location
    
    
    #salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'})
                 
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_sarary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if is_check_sarary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = int(salary[0].getText() \
                                        .replace(u'\xa0', u''))
        elif is_check_sarary == 'от':
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = None
        else:
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = int(salary[2].getText() \
                                         .replace(u'\xa0', u''))           
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    # link
    vacancy_link = item.find_all('div', {'class': 'iJCa5 f-test-vacancy-item _1fma_'})
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru/{vacancy_link }'
    
    # site
    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [708]:
def _parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'clusters':'true', \
        'area':'1', \
        'enable_snippets':'true', \
        'salary':'', \
        'st':'searchVacancy', \
        'text':'Инженер сметчик'
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
    }

    link = 'https://hh.ru/search/vacancy/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-page'})
        if not page_block:
            last_page = int('1')
        else:
            last_page = int(page_block.find_all('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})[-2].getText())
        
    
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(_parser_item_hh(item))
                
    return vacancy_date

In [709]:
def _parser_item_hh(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_date['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}).getText()
    
    vacancy_date['company_name'] = company_name
    
    # city
    city = item.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}) \
                .getText() \
                .split(', ')[0]
    
    vacancy_date['city'] = city
    
    
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        
        salary_currency = salary[2]
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    # link
    
    vacancy_link = item.find('a', {'class': 'bloko-link HH-LinkModifier'}) \
                        
    
    vacancy_date['vacancy_link'] = vacancy_link 
    
    
    # site
    vacancy_date['site'] = 'hh.ru'
    
    return vacancy_date

In [710]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(_parser_hh(vacancy))
    vacancy_date.extend(_parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [711]:
vacancy = 'Инженер сметчик == inzhener smetchik '
df = parser_vacancy(vacancy)

In [712]:
df[40:52]

,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
40,Инженер-сметчик по общестроительным работам,ООО KESZ,Москва,NaN,NaN,None,[Инженер-сметчик по общестроительным работам],hh.ru
41,Инженер-сметчик (электрооборудование),АО Институт Гидропроект,Москва,NaN,NaN,None,[Инженер-сметчик (электрооборудование)],hh.ru
42,Инженер-сметчик,ООО АТОМСТРОЙРЕЗЕРВ,Москва,NaN,NaN,None,[Инженер-сметчик],hh.ru
43,Инженер-сметчик в сфере малоэтажного домострое...,ГК Строй Коттедж,Москва,80000.0,NaN,руб.,[Инженер-сметчик в сфере малоэтажного домостро...,hh.ru
44,Инженер ПТО / Сметчик,ООО СК СитиАльп,Москва,80000.0,90000.0,руб.,[Инженер ПТО / Сметчик],hh.ru
45,Инженер-сметчик,АО ММК МОСИНТРАСТ,Москва,70000.0,90000.0,руб.,[Инженер-сметчик],hh.ru
46,Инженер-сметчик,ООО СК Эпсилон,Москва,80000.0,NaN,руб.,[Инженер-сметчик],hh.ru
47,Инженер-сметчик,ООО СПК-Инжиниринг,Москва,90000.0,NaN,руб.,[Инженер-сметчик],hh.ru
48,Инженер-сметчик,АЯКС,Москва,80000.0,NaN,руб.,[Инженер-сметчик],hh.ru
49,Инженер-Проектировщик/Сметчик,ООО Дс Ерс-Контрактор,Москва,60000.0,100000.0,руб.,[Инженер-Проектировщик/Сметчик],hh.ru
